In [13]:
import os
import json
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from typing import Dict, List, Tuple
from pathlib import Path

# Get the results directory
try:
    # When running as a script
    RESULTS_DIR = Path(__file__).parent.parent
    ANALYSIS_DIR = Path(__file__).parent
except NameError:
    # When running in Jupyter notebook
    RESULTS_DIR = Path.cwd().parent
    ANALYSIS_DIR = Path.cwd()

def load_matches_from_folder(folder_path: str) -> List[Tuple[str, str, str]]:
    """Load all matches from a matches-cached folder."""
    matches = []
    matches_path = Path(folder_path) / 'matches-cached'
    if not matches_path.exists():
        return matches
        
    for filename in matches_path.iterdir():
        if filename.suffix == '.json':
            with open(filename) as f:
                match_data = json.load(f)
                matches.append((
                    match_data['winningTeam'],
                    match_data['losingTeam'],
                    filename.name
                ))
    return matches

def analyze_model_predictions(real_world_path: str, model_path: str, stage: str) -> Dict:
    """Compare model predictions with real world results for a given stage."""
    real_matches = load_matches_from_folder(real_world_path / stage)
    model_matches = load_matches_from_folder(model_path / stage)
    
    if not real_matches or not model_matches:
        return {
            'correct': 0,
            'total': 0,
            'accuracy': 0,
            'correct_matches': [],
            'wrong_matches': []
        }

    correct = 0
    correct_matches = []
    wrong_matches = []
    analyzed_matches = 0
    
    # Create dictionaries for both filename and team combination lookups
    real_results_by_file = {filename: (winner, loser) for winner, loser, filename in real_matches}
    real_results_by_teams = {}
    for winner, loser, filename in real_matches:
        match_key = f"{winner}-{loser}"
        alt_key = f"{loser}-{winner}"
        real_results_by_teams[match_key] = (winner, loser, filename)
        real_results_by_teams[alt_key] = (winner, loser, filename)
    
    # Compare predictions
    for model_winner, model_loser, model_filename in model_matches:
        match_key = f"{model_winner}-{model_loser}"
        alt_key = f"{model_loser}-{model_winner}"
        
        # Check if this match combination actually happened (regardless of filename)
        if match_key in real_results_by_teams or alt_key in real_results_by_teams:
            analyzed_matches += 1
            real_match = real_results_by_teams.get(match_key) or real_results_by_teams.get(alt_key)
            real_winner = real_match[0]  # First element is the winner
            
            if real_winner == model_winner:
                correct += 1
                correct_matches.append(f"{model_winner} vs {model_loser}")
            else:
                wrong_matches.append(f"{model_winner} vs {model_loser} (actual: {real_winner})")

    accuracy = (correct / analyzed_matches * 100) if analyzed_matches > 0 else 0

    return {
        'correct': correct,
        'total': analyzed_matches,
        'accuracy': accuracy,
        'correct_matches': correct_matches,
        'wrong_matches': wrong_matches
    }

def generate_analysis():
    stages = ['stage1', 'stage2', 'stage3', 'playoffs']
    models = [
        d for d in RESULTS_DIR.iterdir() 
        if d.is_dir() 
        and d.name != '0real-world'
        and d.name != 'stats'
        and d.name != 'analysis'
        and not d.name.startswith('.')
    ]
    
    # Store results for plotting
    plot_data = {model.name: [] for model in models}
    
    # Generate analysis markdown file
    analysis_file = ANALYSIS_DIR / 'analysis-per-match.md'
    with open(analysis_file, 'w') as f:
        f.write("# CS2 Match Prediction Analysis\n\n")
        
        for model in models:
            f.write(f"## {model.name}\n\n")
            
            for stage in stages:
                results = analyze_model_predictions(
                    RESULTS_DIR / '0real-world',
                    model,
                    stage
                )
                plot_data[model.name].append(results['accuracy'])
                
                f.write(f"### {stage}\n")
                f.write(f"Accuracy: {results['accuracy']:.2f}% ({results['correct']}/{results['total']} matches)\n\n")
                
                if results['correct_matches']:
                    f.write("Correct Predictions:\n")
                    for match in results['correct_matches']:
                        f.write(f"- {match}\n")
                    f.write("\n")
                
                if results['wrong_matches']:
                    f.write("Wrong Predictions:\n")
                    for match in results['wrong_matches']:
                        f.write(f"- {match}\n")
                    f.write("\n")
            
            f.write("---\n\n")
    
    # Styling constants
    colors = ['#2ecc71', '#e74c3c', '#3498db', '#f1c40f', '#9b59b6', '#1abc9c']
    plt.style.use('default')

    # 1. Line Chart
    plt.figure(figsize=(12, 7))
    for i, (model_name, accuracies) in enumerate(plot_data.items()):
        line = plt.plot(stages, accuracies, 
                marker='o',
                linewidth=3,
                markersize=10,
                label=model_name,
                color=colors[i % len(colors)])
        
        # Add accuracy numbers above each point
        for x, y in enumerate(accuracies):
            # Adjust vertical position if accuracy is high
            y_offset = -15 if y > 90 else 10  # Put annotation below point if accuracy > 90%
            
            plt.annotate(f'{y:.1f}%', 
                    (x, y),
                    textcoords="offset points", 
                    xytext=(0, y_offset),
                    ha='center',
                    va='bottom' if y_offset > 0 else 'top',
                    fontsize=9)

    plt.title('Model Prediction Accuracy Across Stages - Analysis per matches that happened in the championship', pad=20, fontsize=14)
    plt.xlabel('Stage', fontsize=12)
    plt.ylabel('Accuracy (%)', fontsize=12)
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.xticks(range(len(stages)), stages, rotation=45)

    # Adjust y-axis limit for annotations
    max_accuracy = max(max(accuracies) for accuracies in plot_data.values())
    plt.ylim(0, max(100, max_accuracy + 15))

    plt.tight_layout()
    plt.savefig(ANALYSIS_DIR / 'analysis-per-match-line.png', dpi=300, bbox_inches='tight')
    plt.close()

    # 3. Faceted Line Charts
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    axes = axes.ravel()

    for i, (model_name, accuracies) in enumerate(plot_data.items()):
        ax = axes[i]
        ax.plot(stages, accuracies,
                marker='o',
                linewidth=3,
                markersize=10,
                color=colors[i % len(colors)])
        
        # Set y-axis limit with some padding for annotations
        max_accuracy = max(accuracies)
        y_limit = max(100, max_accuracy + 15)  # Add padding for annotations
        ax.set_ylim(0, y_limit)
        
        # Add accuracy numbers above each point
        for x, y in enumerate(accuracies):
            # Adjust vertical position if accuracy is high
            y_offset = -15 if y > 90 else 10  # Put annotation below point if accuracy > 90%
            
            ax.annotate(f'{y:.1f}%', 
                    (x, y),
                    textcoords="offset points", 
                    xytext=(0, y_offset),
                    ha='center',
                    va='bottom' if y_offset > 0 else 'top',
                    fontsize=9)
        
        ax.set_title(model_name, fontsize=12, pad=20)  # Added padding to title
        ax.grid(True, linestyle='--', alpha=0.7)
        ax.set_xticks(range(len(stages)))
        ax.set_xticklabels(stages, rotation=45)
        ax.set_ylabel('Accuracy (%)')

    # Hide empty subplots if any
    for j in range(i + 1, len(axes)):
        axes[j].set_visible(False)

    plt.suptitle('Individual Model Performance Across Stages - Analysis per matches that happened in the championship', fontsize=14, y=1.02)
    plt.tight_layout()
    plt.savefig(ANALYSIS_DIR / 'analysis-per-match-faceted.png', dpi=300, bbox_inches='tight')
    plt.close()

    # 4. Heatmap
    plt.figure(figsize=(10, 8))
    accuracy_matrix = np.array([accuracies for accuracies in plot_data.values()])

    sns.heatmap(accuracy_matrix,
                xticklabels=stages,
                yticklabels=plot_data.keys(),
                annot=True,
                fmt='.1f',  # Format to 1 decimal place
                annot_kws={'size': 9},
                cmap='RdYlGn',
                center=50,
                vmin=0,
                vmax=100,
                cbar_kws={'label': 'Accuracy (%)'}
                )

    # Add % symbol to annotations after they're created
    for t in plt.gca().texts:
        t.set_text(t.get_text() + '%')

    plt.title('Accuracy Heatmap Across Stages and Models - Analysis per matches that happened in the championship', pad=20, fontsize=14)
    plt.xlabel('Stage', fontsize=12)
    plt.ylabel('Model', fontsize=12)
    plt.tight_layout()
    plt.savefig(ANALYSIS_DIR / 'analysis-per-match-heatmap.png', dpi=300, bbox_inches='tight')
    plt.close()


if __name__ == "__main__":
    generate_analysis()